In [1]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('../data/cleaned_security_incidents.csv')


In [2]:
print(df.country)


0                     Rwanda
1               Sierra Leone
2                     Rwanda
3                     Rwanda
4                     Rwanda
                ...         
3952                   Sudan
3953                DR Congo
3954                 Nigeria
3955    Syrian Arab Republic
3956                 Nigeria
Name: country, Length: 3957, dtype: object


In [15]:

# 1. Map Country to ISO codes (use your own `iso_codes` mapping if you already have it)
# Or use `pycountry` to auto-map (optional)
import pycountry

def get_iso_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df['ISO'] = df['country'].apply(get_iso_code)

# 2. Filter by years in your range
df = df[(df['year'] >= 1997) & (df['year'] <= 2025)]

# 3. Group by Country and Year (summing Total.killed per country-year if needed)
df_grouped = df.groupby(['country', 'ISO', 'year'], as_index=False)['total_affected'].sum()

# 4. Plot the choropleth map
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total_affected',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Killed by Country (1997–2025)'
)

fig.update_layout(
    title_x=0.5,
    geo=dict(showframe=False, showcoastlines=True),
    width=1000,
    height=600,
    template='plotly_white',
    coloraxis_colorbar=dict(title="Total Killed")
)

fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Total killed: %{z}<extra></extra>"
)

fig.show()


## Visualization with context / timeline on top: 

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pycountry

# --- 1. ISO Mapping ---
def get_iso_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df['ISO'] = df['country'].apply(get_iso_code)

# --- 2. Filter years ---
df = df[(df['year'] >= 1997) & (df['year'] <= 2025)]

# --- 3. Group data ---
df_grouped = df.groupby(['country', 'ISO', 'year'], as_index=False)['total_affected'].sum()

# --- 4. Create base choropleth ---
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total_affected',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Affected by Country (1997–2025)',
    range_color=[0, df_grouped['total_affected'].max()],  # FIXED SCALE
)

# --- 5. Customize layout ---
fig.update_layout(
    # Set geographic scope to focus on Europe, Africa, and parts of Asia
    geo=dict(
        scope='world',
        projection=dict(type='natural earth'),
        # Specify the boundaries (longitude and latitude)
        # West: Western Europe (-15), East: India (85), 
        # South: Southern Africa (-40), North: Northern Europe (70)
        lonaxis=dict(range=[-15, 85]),
        lataxis=dict(range=[-40, 70]),
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
    ),
    transition=dict(
        duration=1000,
        # smoother fade duration (in ms)
        easing='cubic-in-out'  # smooth in/out effect
    ),
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {
                'frame': {'duration': 1500, 'redraw': True},  # slower year change
                'fromcurrent': True,
                'transition': {'duration': 1000, 'easing': 'cubic-in-out'}
            }]
        }, {
            'label': 'Pause',
            'method': 'animate',
            'args': [[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }]
        }]
    }],
    sliders=[{
        'active': 0,
        'steps': [
            {
                'method': 'animate',
                'label': str(year),
                'args': [[str(year)], {
                    'frame': {'duration': 1500, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 1000, 'easing': 'cubic-in-out'}
                }]
            } 
            for year in df_grouped['year'].unique()
        ],
        'transition': {'duration': 1000},
        'x': 0.1,
        'len': 0.9
    }]
)

# --- 6. Add annotation frames for 2006–2011 ---
frames_with_annotation = []
for frame in fig.frames:
    year = int(frame.name)
    # Copy the geo settings to each frame
    frame.layout = go.Layout(
        geo=dict(
            scope='world',
            projection=dict(type='natural earth'),
            lonaxis=dict(range=[-15, 85]),
            lataxis=dict(range=[-40, 70]),
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
        )
    )
    
    # Add annotations for specific years
    if 2006 <= year <= 2011:
        frame.layout.annotations = [dict(
            x=0.01,
            y=0.95,
            xref='paper',
            yref='paper',
            text="Height of Afghanistan-US Conflict",
            showarrow=False,
            font=dict(size=12, color='white'),
            bgcolor='rgba(255, 0, 0, 0.6)',
            bordercolor='red',
            borderwidth=2,
            borderpad=4
        )]
    
    frames_with_annotation.append(frame)

fig.frames = frames_with_annotation

# --- 7. Show figure ---
fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'xref' property of layout.shape
        Received value: 'geo'

    The 'xref' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['paper']
      - A string that matches one of the following regular expressions:
            ['^x([2-9]|[1-9][0-9]+)?( domain)?$']

In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pycountry

# --- 1. ISO Mapping ---
def get_iso_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df['ISO'] = df['country'].apply(get_iso_code)

# --- 2. Filter years ---
df = df[(df['year'] >= 1997) & (df['year'] <= 2025)]

# --- 3. Group data ---
df_grouped = df.groupby(['country', 'ISO', 'year'], as_index=False)['total_affected'].sum()

# --- 4. Create base choropleth ---
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total_affected',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Affected by Country (1997–2025)',
    range_color=[0, df_grouped['total_affected'].max()],  # FIXED SCALE
)

# --- 5. Customize layout ---
fig.update_layout(
    transition=dict(
        duration=1000,         # smoother fade duration (in ms)
        easing='cubic-in-out'  # smooth in/out effect
    ),
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {
                'frame': {'duration': 1500, 'redraw': True},  # slower year change
                'fromcurrent': True,
                'transition': {'duration': 1000, 'easing': 'cubic-in-out'}
            }]
        }, {
            'label': 'Pause',
            'method': 'animate',
            'args': [[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }]
        }]
    }],
    sliders=[{
        'active': 0,
        'steps': [
            {
                'method': 'animate',
                'label': str(year),
                'args': [[str(year)], {
                    'frame': {'duration': 1500, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 1000, 'easing': 'cubic-in-out'}
                }]
            } 
            for year in df_grouped['year'].unique()
        ],
        'transition': {'duration': 1000},
        'x': 0.1,
        'len': 0.9
    }]
)

# --- 6. Add annotation frames for 2006–2011 ---
# Add annotation frames for 2006–2011 using geographic reference
frames_with_annotation = []
for frame in fig.frames:
    year = int(frame.name)
    if 2006 <= year <= 2011:
        frame.layout = go.Layout(
            annotations=[dict(
                x=0.01,
                y=0.95,
                xref='paper',
                yref='paper',
                text="Height of Afghanistan-US Conflict",
                showarrow=False,
                font=dict(size=12, color='white'),
                bgcolor='rgba(255, 0, 0, 0.6)',
                bordercolor='red',
                borderwidth=2,
                borderpad=4
            )]
        )
    frames_with_annotation.append(frame)

fig.frames = frames_with_annotation

# --- 7. Show figure ---
fig.show()


In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pycountry

# --- 1. ISO Mapping ---
def get_iso_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df['ISO'] = df['country'].apply(get_iso_code)

# --- 2. Filter years ---
df = df[(df['year'] >= 1997) & (df['year'] <= 2025)]

# --- 3. Group data ---
df_grouped = df.groupby(['country', 'ISO', 'year'], as_index=False)['total_affected'].sum()

# Sort years for proper slider ordering
sorted_years = sorted(df_grouped['year'].unique())

# --- 4. Create base choropleth ---
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total_affected',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Affected by Country (1997–2025)',
    range_color=[0, df_grouped['total_affected'].max()],  # FIXED SCALE
)

# --- 5. Customize layout ---
fig.update_layout(
    # Set geographic scope to focus on Europe, Africa, and parts of Asia
    geo=dict(
        scope='world',
        projection=dict(type='natural earth'),
        # Specify the boundaries (longitude and latitude)
        # West: Western Europe (-15), East: India (85), 
        # South: Southern Africa (-40), North: Northern Europe (70)
        lonaxis=dict(range=[-15, 85]),
        lataxis=dict(range=[-40, 70]),
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
    ),
    transition=dict(
        duration=500,  # Reduced from 1000 to 500 ms
        easing='cubic-in-out'  # smooth in/out effect
    ),
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {
                'frame': {'duration': 800, 'redraw': True},  # Reduced from 1500 to 800 ms
                'fromcurrent': True,
                'transition': {'duration': 500, 'easing': 'cubic-in-out'}  # Reduced from 1000 to 500 ms
            }]
        }, {
            'label': 'Pause',
            'method': 'animate',
            'args': [[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }]
        }]
    }],
    sliders=[{
        'active': 0,
        'steps': [
            {
                'method': 'animate',
                'label': str(year),
                'args': [[str(year)], {
                    'frame': {'duration': 800, 'redraw': True},  # Reduced from 1500 to 800 ms
                    'mode': 'immediate',
                    'transition': {'duration': 500, 'easing': 'cubic-in-out'}  # Reduced from 1000 to 500 ms
                }]
            } 
            for year in sorted_years
        ],
        'transition': {'duration': 1000},
        'x': 0.1,
        'len': 0.9
    }]
)

# --- 6. Add annotation frames for events ---
frames_with_annotation = []
for frame in fig.frames:
    year = int(frame.name)
    # Copy the geo settings to each frame
    frame.layout = go.Layout(
        geo=dict(
            scope='world',
            projection=dict(type='natural earth'),
            lonaxis=dict(range=[-15, 85]),
            lataxis=dict(range=[-40, 70]),
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
        )
    )
    
    # Initialize annotations list for this frame
    annotations = []
    
    # Add Afghanistan-US Conflict annotation for years 2009-2016
    if 2009 <= year <= 2016:
        annotations.append(dict(
            x=0.9,  # Position on right side
            y=0.9,
            xref='paper',
            yref='paper',
            text="Height of Afghanistan-US Conflict",
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='red',
            ax=-80,  # Arrow pointing left toward Afghanistan
            ay=0,
            font=dict(size=8, color='white'),
            bgcolor='rgba(255, 0, 0, 0.6)',
            bordercolor='red',
            borderwidth=2,
            borderpad=4
        ))
    
    # Add South Sudan Civil War annotation for years 2015-2022
    if 2015 <= year <= 2022:
        annotations.append(dict(
            x=0.25,  # Position on left side
            y=0.45,
            xref='paper',
            yref='paper',
            text="South Sudan Civil War<br>2013-2020",
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='red',
            ax=60,  # Arrow pointing right toward Sudan
            ay=0,
            font=dict(size=8, color='white'),
            bgcolor='rgba(255, 0, 0, 0.6)',
            bordercolor='red',
            borderwidth=2,
            borderpad=4
        ))

            # Add Syrian civil war annotation for years 2011-2018
    if 2011 <= year <= 2018:
        annotations.append(dict(
            x=0.27,  # Position on left side
            y=0.9,
            xref='paper',
            yref='paper',
            text="Syrian Civil War<br>2011-2019",
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='red',
            ax=60,  # Arrow pointing right toward Sudan
            ay=0,
            font=dict(size=8, color='white'),
            bgcolor='rgba(255, 0, 0, 0.6)',
            bordercolor='red',
            borderwidth=2,
            borderpad=4
        ))
    
    # Add other conflict events as needed
    # For example, you could add Syrian Civil War, Yemen Crisis, etc.
    
    # Assign annotations to the frame layout if there are any
    if annotations:
        frame.layout.annotations = annotations
    
    frames_with_annotation.append(frame)

fig.frames = frames_with_annotation

# --- 7. Show figure ---
fig.show()

In [4]:
df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))

import pandas as pd
import altair as alt


def create_viz2():
    # Load your data
    df = pd.read_csv('../data/security_incidents.csv')
    df = df.clean_names()
    df.columns = df.columns.str.replace('_', '.', regex=False)
    
    # Create full date column from Year and Month
    df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))

    # Filter out invalid rows
    df = df.dropna(subset=['Date'])

    # Define selectable y-axis variable
    y_dropdown = alt.binding_select(options=['total.affected', 'total.killed', 'total.wounded', 'total.kidnapped'],
                                    name='Y-Axis:')
    y_selection = alt.selection_point(fields=['yvar'], bind=y_dropdown, value='total.affected')

    # Melt your dataframe into long format for flexible plotting
    df_long = df.melt(id_vars=['Date', 'country'], value_vars=['total.affected', 'total.killed', 'total.wounded', 'total.kidnapped'], 
                      var_name='yvar', value_name='yval')

    base = alt.Chart(df_long).transform_filter(
        y_selection
    ).mark_circle(opacity=0.6).encode(
        x=alt.X('Date:T', title="Date"),
        y=alt.Y('yval:Q', title=''),
        color=alt.Color('country:N', legend=None),
        tooltip=['country', 'Date', 'yval', 'yvar']
    ).add_params(
        y_selection
    ).properties(
        width=800,
        height=400,
        title="Interactive Timeline of Humanitarian Impacts"
    ).interactive()

    return base

import altair as alt
alt.data_transformers.disable_max_rows()

chart = create_viz2()
chart


/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_97487/1742252758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))
/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_97487/1742252758.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))


alt.Chart(...)